# Importing All The Modules

In [98]:
import json
import pandas as pd
import json
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


# Csv File Reading

In [3]:
data = pd.read_csv('iris.csv')

In [4]:
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
se=data["species"]

In [6]:
data.drop("species",inplace=True,axis=1)

In [7]:
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [102]:
def read_json_file(filename):

    with open(filename, 'r') as json_file:
        config = json.load(json_file)
    return config

In [11]:
def feature_handling():
    Features=config["design_state_data"]["feature_handling"]
    imputation_dict={}
    for i in data.columns:
        if "impute_with" in Features[str(i)]["feature_details"]:
            Imputation_with=Features[str(i)]["feature_details"]["impute_with"]
            if Imputation_with =="Average of values":
                imputation_dict[i]="mean"
            else:
                imputation_dict[i]=Features[i]["feature_details"]["impute_value"]
    return imputation_dict
        
        
    

In [12]:
def imputation(imputation_values):
    for i in imputation_values:
        if imputation_values[i]=="mean":
            mean_value = data[i].mean()
            data[i].fillna(mean_value, inplace=True)
        else:
            data[i].fillna(int(imputation_values[i]), inplace=True)
    

In [14]:
def feature_reduction_technique():
    feature_reduction=config["design_state_data"]["feature_reduction"]
    return feature_reduction["feature_reduction_method"],feature_reduction
    
    

In [15]:
def Tree_Base(data_dic):
    num_features_to_keep = int(data_dic['num_of_features_to_keep'])
    num_trees = int(data_dic['num_of_trees'])
    depth_of_trees = int(data_dic['depth_of_trees'])
    model = RandomForestClassifier(n_estimators=num_trees, max_depth=depth_of_trees)
    model.fit(data, se)
    feature_importances = model.feature_importances_


In [16]:
def corelation():
    target_col = se
    correlation_matrix = data.corrwith(target_col)
    abs_correlation = correlation_matrix.abs()
    threshold = 0.2 
    selected_features = abs_correlation[abs_correlation > threshold].index.tolist()
    data_imputed_corr = data[selected_features]

In [25]:
def pca():
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(data)
    pca = PCA()
    X_pca = pca.fit_transform(X_standardized)
    explained_variance_ratio = pca.explained_variance_ratio_
    print(explained_variance_ratio)
    

In [36]:
def selected_models():
    models_selection_dict={}
    for i in config["design_state_data"]['algorithms']:
        models_selection_dict[i]=config["design_state_data"]['algorithms'][i]["is_selected"]
    return models_selection_dict

In [87]:
def random_forest_classifier_regressor():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
        }
    try:
        model = RandomForestClassifier()

        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

    except:
        model = RandomForestRegressor()

        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Accuracy: {accuracy:.2f}")

In [71]:
def gbt_classifier_regressor(methods):
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
        }
    if methods=="Regression":
        
        model = GradientBoostingRegressor()
    else:
        model=GradientBoostingClassifier()

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Accuracy: {accuracy:.2f}")

In [56]:
def logistic_regression():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    logistic_params = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100]
    }
    logistic_model = LogisticRegression(max_iter=1000)
    logistic_grid_search = GridSearchCV(logistic_model, logistic_params, cv=5)
    logistic_grid_search.fit(X_train, y_train)
    logistic_best_model = logistic_grid_search.best_estimator_
    logistic_y_pred = logistic_best_model.predict(X_test)
    logistic_accuracy = accuracy_score(y_test, logistic_y_pred)
    print(f"Logistic Regression Accuracy: {logistic_accuracy:.2f}")


In [97]:
def ridge_regression():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    ridge_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
    }
    ridge_model = Ridge()
    ridge_grid_search = GridSearchCV(ridge_model, ridge_params, cv=5)
    ridge_grid_search.fit(X_train, y_train)
    ridge_best_model = ridge_grid_search.best_estimator_
    ridge_y_pred = ridge_best_model.predict(X_test)
    ridge_y_pred = np.argmax(ridge_y_pred, axis=1)  
    ridge_accuracy = accuracy_score(y_test, ridge_y_pred)
    print(f"Ridge Regression Accuracy: {ridge_accuracy:.2f}")


In [ ]:
def lasso_regression():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    lasso_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
    }
    lasso_model = Lasso()
    lasso_grid_search = GridSearchCV(lasso_model, lasso_params, cv=5)
    lasso_grid_search.fit(X_train, y_train)
    lasso_best_model = lasso_grid_search.best_estimator_
    lasso_y_pred = lasso_best_model.predict(X_test)
    lasso_y_pred = np.argmax(lasso_y_pred, axis=1)  # Convert to class labels
    lasso_accuracy = accuracy_score(y_test, lasso_y_pred)
    print(f"Lasso Regression Accuracy: {lasso_accuracy:.2f}")


In [ ]:
def elsatic_net():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    elastic_net_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.5, 0.9]
    }
    elastic_net_model = ElasticNet()
    elastic_net_grid_search = GridSearchCV(elastic_net_model, elastic_net_params, cv=5)
    elastic_net_grid_search.fit(X_train, y_train)
    elastic_net_best_model = elastic_net_grid_search.best_estimator_
    elastic_net_y_pred = elastic_net_best_model.predict(X_test)
    elastic_net_y_pred = np.argmax(elastic_net_y_pred, axis=1)  # Convert to class labels
    elastic_net_accuracy = accuracy_score(y_test, elastic_net_y_pred)
    print(f"Elastic Net Regression Accuracy: {elastic_net_accuracy:.2f}")


In [59]:
def svc():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }
    svm_model = SVC()
    svm_grid_search = GridSearchCV(svm_model, svm_params, cv=5)
    svm_grid_search.fit(X_train, y_train)
    svm_best_model = svm_grid_search.best_estimator_
    svm_y_pred = svm_best_model.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_y_pred)
    print(f"SVM Accuracy: {svm_accuracy:.2f}")


In [64]:
def xg_boost(methods="Regression"):
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    xgboost_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
    }
    if methods=="Regression":
        xgboost_model = XGBRegressor()
    else:
        xgboost_model = XGBClassifier()

    xgboost_grid_search = GridSearchCV(xgboost_model, xgboost_params, cv=5)
    xgboost_grid_search.fit(X_train, y_train)
    xgboost_best_model = xgboost_grid_search.best_estimator_
    xgboost_y_pred = xgboost_best_model.predict(X_test)
    xgboost_accuracy = accuracy_score(y_test, xgboost_y_pred)

In [74]:
def decision_tree_rgressor():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    dt_regressor_params = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
    dt_regressor = DecisionTreeRegressor()
    dt_regressor_grid_search = GridSearchCV(dt_regressor, dt_regressor_params, cv=5, scoring='neg_mean_squared_error')
    dt_regressor_grid_search.fit(X_train, y_train)
    dt_regressor_best_model = dt_regressor_grid_search.best_estimator_
    dt_regressor_y_pred = dt_regressor_best_model.predict(X_test)
    mse_regressor = mean_squared_error(y_test, dt_regressor_y_pred)

    # Print best parameters and mean squared error for DecisionTreeRegressor
    print("DecisionTreeRegressor")
    print(f"Best Parameters: {dt_regressor_grid_search.best_params_}")
    print(f"Mean Squared Error: {mse_regressor:.2f}")

In [76]:
def decision_tree_classifier():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    dt_classifier_params = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
    dt_classifier = DecisionTreeClassifier()
    dt_classifier_grid_search = GridSearchCV(dt_classifier, dt_classifier_params, cv=5)
    dt_classifier_grid_search.fit(X_train, y_train)
    dt_classifier_best_model = dt_classifier_grid_search.best_estimator_
    dt_classifier_y_pred = dt_classifier_best_model.predict(X_test)
    accuracy_classifier = accuracy_score(y_test, dt_classifier_y_pred)

    # Print best parameters and accuracy for DecisionTreeClassifier
    print("DecisionTreeClassifier")
    print(f"Best Parameters: {dt_classifier_grid_search.best_params_}")
    print(f"Accuracy: {accuracy_classifier:.2f}")


In [ ]:
def linear_regression():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    linear_regressor = LinearRegression()

    params = {
        'fit_intercept': [True, False],
        'normalize': [True, False]
    }

    grid_search = GridSearchCV(linear_regressor, params, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    best_linear_model = grid_search.best_estimator_

    linear_y_pred = best_linear_model.predict(X_test)

    print("Linear Regression")
    print(f"Best Parameters: {grid_search.best_params_}")


In [78]:
def knn():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    knn_params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}
    knn_classifier = KNeighborsClassifier()
    knn_grid_search = GridSearchCV(knn_classifier, knn_params, cv=5)
    knn_grid_search.fit(X_train, y_train)
    knn_best_model = knn_grid_search.best_estimator_
    knn_y_pred = knn_best_model.predict(X_test)
    accuracy_knn = accuracy_score(y_test, knn_y_pred)

    # Print best parameters and accuracy for KNN Classifier
    print("KNN Classifier")
    print(f"Best Parameters: {knn_grid_search.best_params_}")
    print(f"Accuracy: {accuracy_knn:.2f}")


In [80]:
def extratree_classifier():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)

    extra_trees_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
    extra_trees_classifier = ExtraTreesClassifier()
    extra_trees_grid_search = GridSearchCV(extra_trees_classifier, extra_trees_params, cv=5)
    extra_trees_grid_search.fit(X_train, y_train)
    extra_trees_best_model = extra_trees_grid_search.best_estimator_
    extra_trees_y_pred = extra_trees_best_model.predict(X_test)
    accuracy_extra_trees = accuracy_score(y_test, extra_trees_y_pred)

    # Print best parameters and accuracy for Extra Trees Classifier
    print("Extra Trees Classifier")
    print(f"Best Parameters: {extra_trees_grid_search.best_params_}")
    print(f"Accuracy: {accuracy_extra_trees:.2f}")


In [79]:
def sgd(methods):
    sgd_params = {
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01]}
    if methods =='Regression':
        sgd_model = SGDRegressor(max_iter=100)
    else:
        sgd_model=SGDClassifier(max_iter=100)
    sgd_grid_search = GridSearchCV(sgd_model, sgd_params, cv=5)
    sgd_grid_search.fit(X_train, y_train)
    sgd_best_model = sgd_grid_search.best_estimator_
    sgd_y_pred = sgd_best_model.predict(X_test)
    accuracy_sgd = accuracy_score(y_test, sgd_y_pred)
    print(f"Accuracy: {accuracy_sgd:.2f}")

In [ ]:
def nural_network():
    X_train, X_test, y_train, y_test = train_test_split(data,se, test_size=0.2, random_state=42)
    nn_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    nn_classifier.fit(X_train, y_train)
    nn_y_pred = nn_classifier.predict(X_test)
    accuracy_nn = accuracy_score(y_test, nn_y_pred)

    # Print accuracy for Neural Network Classifier
    print(f"Neural Network Accuracy: {accuracy_nn:.2f}")

# Read Json File

In [106]:
config=read_json_file("data.json")

In [107]:
target_col = config["design_state_data"]["target"]
regression_type = target_col["prediction_type"]
print("The Type is::-->",regression_type)

The Type is::--> Regression


In [108]:
methods=regression_type

# Finding the Imputation Values

In [109]:
Imputation_values=feature_handling()
print("Imputation Values:->",Imputation_values)

Imputation Values:-> {'sepal_length': 'mean', 'sepal_width': -1, 'petal_length': 'mean', 'petal_width': -2}


# Using The Imputation Values We Will Do The Imputation

In [110]:
Imputation=imputation(Imputation_values)

# Finding The Feature Reduction Technique Method

In [111]:
method,feature_reduction=feature_reduction_technique()
print("The method of Feature Reduction Technique--->",method)

The method of Feature Reduction Technique---> Tree-based


In [112]:
if method =='Tree-based':
    Tree_Base(feature_reduction)
elif method =='Correlation with target':
    corelation()
elif method =='pca':
    pca()
else:
    pass
    

# Finding Which Models To Run

In [115]:
selcted_models_dic=selected_models()


In [117]:
print("The Models To Run--->",selcted_models_dic)

The Models To Run---> {'RandomForestClassifier': False, 'RandomForestRegressor': True, 'GBTClassifier': False, 'GBTRegressor': False, 'LinearRegression': False, 'LogisticRegression': False, 'RidgeRegression': False, 'LassoRegression': False, 'ElasticNetRegression': False, 'xg_boost': False, 'DecisionTreeRegressor': False, 'DecisionTreeClassifier': False, 'SVM': False, 'SGD': False, 'KNN': False, 'extra_random_trees': False, 'neural_network': False}


# Running The Particular Models

In [113]:
for i in selcted_models_dic:
    if i == "RandomForestClassifier" and selcted_models_dic[i]==True:
        print("******** Results For RandomForestClassifier **********")
        random_forest_classifier_regressor()
    elif i == "RandomForestRegressor" and selcted_models_dic[i]==True:
        print("******** Results For RandomForestRegressor **********")
        random_forest_classifier_regressor()
    elif i == "GBTClassifier" and selcted_models_dic[i]==True:
        print("******** Results For GBTClassifier **********")
        gbt_classifier_regressor(methods)
    elif i == "GBTRegressor" and selcted_models_dic[i]==True:
        print("******** Results For GBTRegressor **********")
        gbt_classifier_regressor(methods)
    elif i == "LogisticRegression" and selcted_models_dic[i]==True:
        print("******** Results For LogisticRegression **********")
        logistic_regression()
    elif i == "RidgeRegression" and selcted_models_dic[i]==True:
        print("******** Results For RidgeRegression **********")
        ridge_regression()
    elif i == "LassoRegression" and selcted_models_dic[i]==True:
        print("******** Results For LassoRegression **********")
        lasso_regression()
    elif i == "ElasticNetRegression" and selcted_models_dic[i]==True:
        print("******** Results For ElasticNetRegression **********")
        elsatic_net()
    elif i == "SVM" and selcted_models_dic[i]==True:
        print("******** Results For SVM **********")
        svc()
    elif i == "SGD" and selcted_models_dic[i]==True:
        print("******** Results For SGD **********")
        sgd(methods)
    elif i == "xg_boost" and selcted_models_dic[i]==True:
        print("******** Results For xg_boost **********")
        xg_boost(methods)
    elif i == "DecisionTreeRegressor" and selcted_models_dic[i]==True:
        print("******** Results For DecisionTreeRegressor **********")
        decision_tree_rgressor()
    elif i == "DecisionTreeClassifier" and selcted_models_dic[i]==True:
        print("******** Results For DecisionTreeClassifier **********")
        decision_tree_classifier()
    elif i == "KNN" and selcted_models_dic[i]==True:
        print("******** Results For KNN **********")
        knn()
    elif i == "extra_random_trees" and selcted_models_dic[i]==True:
        print("******** Results For extra_random_trees **********")
        extratree_classifier()
    elif i == "neural_network" and selcted_models_dic[i]==True:
        print("******** Results For neural_network **********")
        nural_network()
    elif i == "LinearRegression" and selcted_models_dic[i]==True:
        print("******** Results For LinearRegression **********")
        linear_regression()
        
        



******** Results For RandomForestRegressor **********
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy: 1.00
